# OSM Folium Visualizations

Data Exploration: folium

In [50]:
# imports 

import pandas as pd
import folium # map visuals
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from geopy.distance import geodesic # distancce analysis

## Perform data cleaning & exploration

In [51]:
slofood_parquet = pd.read_parquet("slofood_cleaned.parquet")
davisfood_parquet = pd.read_parquet("davisfood_cleaned.parquet")
athensfood_parquet = pd.read_parquet("athensfood_cleaned.parquet")

## mark places for each town

Since I'm working with map/location data, I would obviously need to try some map visuals. 

Here are simple map visualizations of all the food places plotted out for each town.

In [52]:
slofood_map = folium.Map(location=[35.3050, -120.6625], zoom_start=12)

# iterate thru parquet & add markers for each location
for _, row in slofood_parquet.iterrows():
    lat = row['lat']
    lon = row['lon']
    name = row['name'] 
    
    # add marker with popup showing name of location
    folium.Marker(
        location=[lat, lon],
        popup=name  # show name of place when marker clicked
    ).add_to(slofood_map)

# slofood_map.save('slofood_map.html')  # save
slofood_map

In [53]:
davisfood_map = folium.Map(location=[38.5382, -121.7617], zoom_start=12)

# iterate thru parquet & add markers for each location
for _, row in davisfood_parquet.iterrows():
    lat = row['lat']
    lon = row['lon']
    name = row['name'] 
    
    # add marker with popup showing name of location
    folium.Marker(
        location=[lat, lon],
        popup=name  # show name of place when marker clicked
    ).add_to(davisfood_map)

# davisfood_map.save('davisfood_map.html')  # save
davisfood_map

In [54]:
athensfood_map = folium.Map(location=[33.9519, -83.3576], zoom_start=12)

# iterate thru parquet & add markers for each location
for _, row in athensfood_parquet.iterrows():
    lat = row['lat']
    lon = row['lon']
    name = row['name'] 
    
    # add marker with popup showing name of location
    folium.Marker(
        location=[lat, lon],
        popup=name  # show name of place when marker clicked
    ).add_to(athensfood_map)

# athensfood_map.save('athensfood_map.html')  # save
athensfood_map

## heat map for each town

The markers above didn't tell much, so a heatmap could do a better job on telling me the distributions of food spots in each town.

In [55]:
slofood_heatmap = folium.Map(location=[35.2828, -120.6596], zoom_start=13)
heat_data = slofood_parquet[['lat', 'lon']].dropna().values.tolist() # extract out lat & long
HeatMap(heat_data).add_to(slofood_heatmap) # heatmap layer

slofood_heatmap

In [56]:
davisfood_heatmap = folium.Map(location=[38.5382, -121.7617], zoom_start=13)
heat_data = davisfood_parquet[['lat', 'lon']].dropna().values.tolist() # extract out lat & long
HeatMap(heat_data).add_to(davisfood_heatmap) # heatmap layer

davisfood_heatmap

In [57]:
athensfood_heatmap = folium.Map(location=[33.9519, -83.3576], zoom_start=12)
heat_data = athensfood_parquet[['lat', 'lon']].dropna().values.tolist() # extract out lat & long
HeatMap(heat_data).add_to(athensfood_heatmap) # heatmap layer

athensfood_heatmap

## distance analysis

I just wanted to look at how far some brands were from campus
* Starbucks
* Subway
* Taco Bell

In [58]:
# University coordinates (latitude, longitude)
universities = {
    "San Luis Obispo": (35.3004, -120.6625),  # Cal Poly SLO
    "Davis": (38.5449, -121.7405),  # UC Davis
    "Athens": (33.9500, -83.3740)  # University of Georgia
}

# Dictionary to store city dataframes
city_data = {
    "San Luis Obispo": slofood_parquet,
    "Davis": davisfood_parquet,
    "Athens": athensfood_parquet
}

# folium map for distances from each brand location to its respective university
def map_brand_distance(brand_name):
    
    brand_map = folium.Map(location=[36.5, -100], zoom_start=5) # center around US (can change for each uni)

    def analyze_brand(city, food_df, brand):

        university_loc = universities[city]
        
        brand_df = food_df[food_df['brand'] == brand] # filter brand locations
        
        for _, row in brand_df.iterrows():
            brand_loc = (row['lat'], row['lon'])
            distance = geodesic(university_loc, brand_loc).miles
            
            # mark each locatoin
            folium.Marker(
                location=brand_loc, 
                popup=f"{brand} in {city} ({distance:.2f} mi)",  
                icon=folium.Icon(color="green", icon="info-sign")
            ).add_to(brand_map)
            
            # mark university (only add once per city)
            folium.Marker(
                location=university_loc,
                popup=f"{city} University",
                icon=folium.Icon(color="red", icon="university")
            ).add_to(brand_map)
            
            # draw line from brand location to uni
            folium.PolyLine([university_loc, brand_loc], color="blue", weight=2.5,
                            tooltip=f"{distance:.2f} mi").add_to(brand_map)

            print(f"{brand} in {city} -> {city} University: {distance:.2f} miles")


    # run for all cities
    for city, food_df in city_data.items():
        analyze_brand(city, food_df, brand_name)

    return brand_map

In [59]:
map_brand_distance('Starbucks')

Starbucks in San Luis Obispo -> San Luis Obispo University: 3.62 miles
Starbucks in San Luis Obispo -> San Luis Obispo University: 3.70 miles
Starbucks in San Luis Obispo -> San Luis Obispo University: 0.21 miles
Starbucks in San Luis Obispo -> San Luis Obispo University: 0.66 miles
Starbucks in San Luis Obispo -> San Luis Obispo University: 1.34 miles
Starbucks in San Luis Obispo -> San Luis Obispo University: 2.69 miles
Starbucks in San Luis Obispo -> San Luis Obispo University: 0.20 miles
Starbucks in San Luis Obispo -> San Luis Obispo University: 3.55 miles
Starbucks in San Luis Obispo -> San Luis Obispo University: 2.09 miles
Starbucks in San Luis Obispo -> San Luis Obispo University: 1.35 miles
Starbucks in Davis -> Davis University: 2.50 miles
Starbucks in Davis -> Davis University: 1.09 miles
Starbucks in Davis -> Davis University: 1.96 miles
Starbucks in Davis -> Davis University: 0.86 miles
Starbucks in Davis -> Davis University: 8.08 miles
Starbucks in Davis -> Davis Univers

^^ Starbucks is the top brand for both SLO and Davis. I can see that in SLO, they have the most Starbucks, and they are all within around 3.5 miles from campus. For Davis and Athens, there is still a Starbucks close by campus, but there are also some others which are much further.

In [60]:
map_brand_distance('Subway')

Subway in San Luis Obispo -> San Luis Obispo University: 2.20 miles
Subway in San Luis Obispo -> San Luis Obispo University: 0.06 miles
Subway in San Luis Obispo -> San Luis Obispo University: 2.91 miles
Subway in San Luis Obispo -> San Luis Obispo University: 0.50 miles
Subway in San Luis Obispo -> San Luis Obispo University: 3.33 miles
Subway in Davis -> Davis University: 0.14 miles
Subway in Davis -> Davis University: 2.57 miles
Subway in Davis -> Davis University: 1.04 miles
Subway in Davis -> Davis University: 8.19 miles
Subway in Davis -> Davis University: 6.63 miles
Subway in Davis -> Davis University: 1.95 miles
Subway in Athens -> Athens University: 5.86 miles


Subway, the second top brand, appears to be close to campus for SLO and Davis. However, it is much further for Athens.

In [61]:
map_brand_distance('Taco Bell')

Taco Bell in San Luis Obispo -> San Luis Obispo University: 3.63 miles
Taco Bell in San Luis Obispo -> San Luis Obispo University: 0.52 miles
Taco Bell in Davis -> Davis University: 0.14 miles


Suprisingly, there is no Taco Bell in the Athens area, coordinates "-83.5,33.88,-83.3,34.05". There are more Taco Bells for SLO, but the one in Davis is closer to campus.

## misc??

I tried extracting different things from the dataset to look at specifically. However, this can be difficult to visualize since the towns are so spread out, and even within each town, each 'thing' like `cuisine` are distributed sparsely around a town (not like all the italian restaurants are all together).

In [62]:
map_davis = folium.Map(location=[38.5449, -121.7405], zoom_start=13)
marker_cluster = MarkerCluster().add_to(map_davis)

for _, row in davisfood_parquet.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"{row['name']} ({row.get('cuisine', 'Unknown')})",
        icon=folium.Icon(color="blue" if row.get('cuisine') == "fast_food" else "green"),
    ).add_to(marker_cluster)

map_davis

In [63]:
map_combined = folium.Map(location=[35.2493387, -120.6420324], zoom_start=12)

marker_cluster = MarkerCluster().add_to(map_combined)
heat_data_combined = []

# Adding markers and heatmap points
for idx, row in slofood_parquet.iterrows():
    lat = row['lat']
    lon = row['lon']
    cuisine = row['cuisine']
    brand = row['brand']
    name = row['name']

    # Add data for the heatmap
    heat_data_combined.append([lat, lon])

    # Add marker for the cluster
    popup_text = f"Name: {name}<br>Cuisine: {cuisine}<br>Brand: {brand}"
    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_cluster)

# Add HeatMap layer to the map
HeatMap(heat_data_combined).add_to(map_combined)

# Show the map
map_combined


In [64]:
# Define map centered in the middle of all locations
comparison_map = folium.Map(location=[36.5, -100], zoom_start=5)  

# Create a MarkerCluster
marker_cluster = MarkerCluster().add_to(comparison_map)

# Function to add markers
def add_markers(parquet_data, color, city):
    for _, row in parquet_data.iterrows():
        lat, lon, name = row['lat'], row['lon'], row['name']
        folium.Marker(
            location=[lat, lon],
            popup=f"{name} ({city})",
            icon=folium.Icon(color=color)
        ).add_to(marker_cluster)

# Add city data
add_markers(slofood_parquet, "green", "SLO")
add_markers(davisfood_parquet, "blue", "Davis")
add_markers(athensfood_parquet, "red", "Athens")

# Show map
comparison_map


In [65]:
# Create map
heatmap = folium.Map(location=[36.5, -100], zoom_start=5)

# Combine all locations
all_food_locations = (
    slofood_parquet[['lat', 'lon']].values.tolist() +
    davisfood_parquet[['lat', 'lon']].values.tolist() +
    athensfood_parquet[['lat', 'lon']].values.tolist()
)

# Add HeatMap
HeatMap(all_food_locations).add_to(heatmap)

heatmap
